<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/program%20synthesis/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Generate the program based on the question using codeTrans single task training model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_base_program_synthese">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [ ]:
!pip install -q transformers sentencepiece

     |████████████████████████████████| 1.4MB 13.0MB/s 
     |████████████████████████████████| 1.1MB 49.9MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 2.9MB 49.7MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [ ]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_program_synthese"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_program_synthese", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


**3 Give the question for generating the code, parse and tokenize it**

In [ ]:
question = "you are given an array of numbers a and a number b, compute the difference of elements in a and b" #@param {type:"raw"}

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def englishTokenizer(sentence):
    result = []
    tokens = word_tokenize(sentence)
    for t in tokens:
        if( not len(t)>50):
            result.append(t)
    return ' '.join(result)

tokenized_question = englishTokenizer(question)
print("tokenized question: " + tokenized_question)

tokenized question: you are given an array of numbers a and a number b , compute the difference of elements in a and b


**4. Make Prediction**

In [ ]:
pipeline([tokenized_question])

Your max_length is set to 512, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': '[ map a [ partial1 b - ] ]'}]